<a href="https://colab.research.google.com/github/chengwen-zheng/openAi-lab/blob/main/LangChain_02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ChatGPT不会算算术


In [ ]:
%pip install langchain
%pip install openai

from googleapiclient.discovery import build
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
import io
import json
import os
import openai

# Authenticate and create the PyDrive client.
# This only needs to be done once per notebook.
auth.authenticate_user()

# Build the Drive API client
drive_service = build('drive', 'v3')
file_id = '14x1k7ErttIPXRcKN53g_Ogn3zfn6AEM_'
file = drive_service.files().get(fileId=file_id).execute()
content = drive_service.files().get_media(fileId=file_id).execute()

# Read the env config.json. set env for OPENAI_API_KEY
with io.BytesIO(content) as f:
    config = json.load(f)
os.environ["OPENAI_API_KEY"] = config["api_key"];
openai.api_key = os.environ["OPENAI_API_KEY"] ;

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 770.9/770.9 kB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 25.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 kB 1.3 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 10.0 MB/s eta 0:00:00


In [ ]:
import openai, os

openai.api_key = os.environ.get("OPENAI_API_KEY")

from langchain.prompts import PromptTemplate
from langchain.llms import OpenAI
from langchain.chains import LLMChain


llm = OpenAI(model_name="text-davinci-003", max_tokens=2048, temperature=0.5)
multiply_prompt = PromptTemplate(template="请计算一下{question}是多少?", input_variables=["question"])
math_chain = LLMChain(llm=llm, prompt=multiply_prompt, output_key="answer")
answer = math_chain.run({"question": "352乘以493"})
print("OpenAI API 说答案是:", answer)

python_answer = 352 * 493
print("Python 说答案是:", python_answer)

OpenAI API 说答案是: 

352 x 493 = 173936
Python 说答案是: 173536


In [ ]:
multiply_by_python_prompt = PromptTemplate(template="请写一段Python代码，计算{question}?", input_variables=["question"])
math_chain = LLMChain(llm=llm, prompt=multiply_by_python_prompt, output_key="answer")
answer = math_chain.run({"question": "352乘以493"})
print(answer)



print(352 * 493)


In [ ]:
multiply_by_python_prompt = PromptTemplate(template="请写一段Python代码，计算{question}?", input_variables=["question"])
math_chain = LLMChain(llm=llm, prompt=multiply_by_python_prompt, output_key="answer")
answer_code = math_chain.run({"question": "352乘以493"})

from langchain.utilities import PythonREPL
python_repl = PythonREPL()
result = python_repl.run(answer_code)
print(result)

173536



# 通过LLMMathChain算算术


In [ ]:
from langchain import LLMMathChain

llm_math = LLMMathChain(llm=llm, verbose=True)
result = llm_math.run("请计算一下352乘以493是多少?")
print(result)

/usr/local/lib/python3.10/dist-packages/langchain/chains/llm_math/base.py:50: UserWarning: Directly instantiating an LLMMathChain with an llm is deprecated. Please instantiate with llm_chain argument or using the from_llm class method.
  warnings.warn(




> Entering new LLMMathChain chain...
请计算一下352乘以493是多少?
```text
352 * 493
```
...numexpr.evaluate("352 * 493")...

Answer: 173536
> Finished chain.
Answer: 173536


# 通过Google搜索查天气

In [ ]:
from langchain.chains import LLMRequestsChain

template = """在 >>> 和 <<< 直接是来自Google的原始搜索结果.
请把对于问题 '{query}' 的答案从里面提取出来，如果里面没有相关信息的话就说 "找不到"
请使用如下格式：
Extracted:<answer or "找不到">
>>> {requests_result} <<<
Extracted:"""

PROMPT = PromptTemplate(
    input_variables=["query", "requests_result"],
    template=template,
)
requests_chain = LLMRequestsChain(llm_chain = LLMChain(llm=OpenAI(temperature=0), prompt=PROMPT))
question = "今天上海的天气怎么样？"
inputs = {
    "query": question,
    "url": "https://www.google.com/search?q=" + question.replace(" ", "+")
}
result=requests_chain(inputs)
print(result)
print(result['output'])

{'query': '今天上海的天气怎么样？', 'url': 'https://www.google.com/search?q=今天上海的天气怎么样？', 'output': '今天上海的天气为晴朗、无云的天空，空气温度为12-16°C，几乎没有风，最大降水机会为0%。'}
今天上海的天气为晴朗、无云的天空，空气温度为12-16°C，几乎没有风，最大降水机会为0%。


# 通过TransformChain运行自定义函数

In [ ]:
import re

def parse_weather_info(weather_info)->dict:
    # 将天气信息拆分成不同部分
    parts = weather_info.split('; ')
    print(parts)
    # 解析天气
    weather = parts[0].strip()

    # 解析温度范围，并提取最小和最大温度
    temperature_range = parts[1].strip().replace('℃', '').split('～')
    print(temperature_range)
    temperature_min = int(temperature_range[0])
    temperature_max = int(temperature_range[1])

    # 解析风向和风力
    wind_parts = parts[2].split(' ')
    wind_direction = wind_parts[0].strip()
    wind_force = wind_parts[1].strip()
    weather_dict = {
        'weather': weather,
        'temperature': {
            'min': temperature_min,
            'max': temperature_max,
        },
        'wind': {
            'direction': wind_direction,
            'level': wind_force
        }
    }
    return weather_dict

# today the weather is changeed, so the format is incorrect. so params of weather is fixed.
weather_dict = parse_weather_info("小雨; 10℃～15℃; 东北风 风力4-5级")
print(weather_dict)

['小雨', '10℃～15℃', '东北风 风力4-5级']
['10', '15']
{'weather': '小雨', 'temperature': {'min': 10, 'max': 15}, 'wind': {'direction': '东北风', 'level': '风力4-5级'}}


In [ ]:
from langchain.chains import TransformChain, SequentialChain

def transform_func(inputs: dict) -> dict:
    
    text ="小雨; 10℃～15℃; 东北风 风力4-5级"
    return {"weather_info" : parse_weather_info(text)}

transformation_chain = TransformChain(input_variables=["output"], 
                                      output_variables=["weather_info"], transform=transform_func)

final_chain = SequentialChain(chains=[requests_chain, transformation_chain], 
                              input_variables=["query", "url"], output_variables=["weather_info"])
final_result = final_chain.run(inputs)

['小雨', '10℃～15℃', '东北风 风力4-5级']
['10', '15']


# 通过VectorDBQA做问答

In [17]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.text_splitter import SpacyTextSplitter
from langchain import OpenAI, VectorDBQA
# from langchain.document_loaders import TextLoader


data_file_id = "1Z_X79_FrTW69h4SND9BnT9qr3ynsdl5G"
data_file = drive_service.files().get(fileId=data_file_id).execute()
content = drive_service.files().get_media(fileId=data_file_id).execute().decode()
# print(content)

documents = Document(content)

llm = OpenAI(temperature=0)
# loader = TextLoader('./data/ecommerce_faq.txt')
# documents = loader.load()
text_splitter = SpacyTextSplitter(chunk_size=256, pipeline="zh_core_web_sm")
texts = text_splitter.split_documents(documents)

embeddings = OpenAIEmbeddings()
docsearch = FAISS.from_documents(texts, embeddings)

faq_chain = VectorDBQA.from_chain_type(llm=llm, vectorstore=docsearch, verbose=True)

NameError: ignored